# Image Segmentation and Contours

In [1]:
import numpy as np
import cv2

### Contour Extraction

In [2]:
image = cv2.imread("shapes.png")
cv2.imshow("Input Image", image)
cv2.waitKey(0)

# grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# find canny edges
edged = cv2.Canny(gray, 30, 200)
cv2.imshow('Canny Edges', edged)
cv2.waitKey(0)

# finding contours
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.imshow('Canny Edges After Contouring', edged)
cv2.waitKey(0)

print('Number of Contours= '+ str(len(contours)))

# drawl all the contours around the shapes
cv2.drawContours(image, contours, -1, (0,255,0),3) # -1 to draw all
cv2.imshow('Contours', image)
cv2.waitKey(0)

cv2.destroyAllWindows()

Number of Contours= 4


### Contour Sorting by Area

In [5]:
# now we sort the contours by area size

def get_contour_areas(contours):
    # returns the areas of all contours in list
    for cnt in contours:
        all_areas = []
        area  = cv2.contourArea(cnt)
        all_areas.append(area)
        print (area)
    return all_areas

# load image 
image = cv2.imread('shapes.png')
original_image = image

# printing the areas of the contours before sorting
print ("Contours Areas before Sorting")
print (get_contour_areas(contours))

# sort contour areas large to small
sorted_contours = sorted(contours, key=cv2.contourArea, reverse = True)

print ("Contour Areas after Sorting") 
print (get_contour_areas(sorted_contours))

for c in sorted_contours:
    cv2.drawContours(original_image, [c], -1, (255,0,0),3)
    cv2.waitKey()
    cv2.imshow('Contours by Area', original_image)

cv2.waitKey()
cv2.destroyAllWindows()    

Contours Areas before Sorting
17686.0
30012.5
65022.0
59675.0
[59675.0]
Contour Areas after Sorting
65022.0
59675.0
30012.5
17686.0
[17686.0]


### Contour Sorting by Position

In [8]:
# sorting  by position

def x_cord_contour(contours):
    if cv2.contourArea(contours)>10:
        M = cv2.moments(contours)
        return (int(M['m10']/M['m00']))
                               
def label_contour_center(image,c):
    # place red circle at center of contours
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    
    cv2.circle(image, (cx,cy),10,(0,0,255),-1)
    return image

# load our image
image = cv2.imread('shapes.png')
original_image = image.copy()

# compute centroids and draw them on image using loop and previous function
for (i,c) in enumerate(contours):
    orig = label_contour_center(image,c)
    
cv2.imshow("4 - Contour Centers ", image)
cv2.waitKey(0)

# sort contours from left to right
contours_left_to_right = sorted(contours, key = x_cord_contour, reverse = False)

# labeling contours from left to right
for (i,c) in enumerate(contours_left_to_right):
    cv2.drawContours(original_image, [c], -1, (0,0,255),3)
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.putText(original_image, str(i+1), (cx,cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)
    cv2.imshow('6 - Left to Right Contours',original_image)
    cv2.waitKey(0)
    (x,y,w,h) = cv2.boundingRect(c)
    
    # now lets crop each contour
    cropped_contour = original_image[y:y+h, x:x+w]
    image_name = "output_shape_number_"+str(i+1)+".jpg"
    print(image_name)
    cv2.imwrite(image_name, cropped_contour)
    
cv2.destroyAllWindows()

output_shape_number_1.jpg
output_shape_number_2.jpg
output_shape_number_3.jpg
output_shape_number_4.jpg


### Approximating Contours

In [2]:
# approximating contours
image = cv2.imread('house.png')
orig_image = image.copy()
cv2.imshow("Original Image",orig_image)
cv2.waitKey()

# grayscale and binarize
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(image_gray, 127, 255, cv2.THRESH_BINARY_INV)

# find contours
contours, heirarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# iterate through each contours and add a bounding box
for c in contours:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(orig_image, (x,y), (x+w,y+h), (0,0,255),2)
    cv2.imshow('Bounding Rectangle', orig_image)
cv2.waitKey(0)

for c in contours:
    # calculate accuracy as a percentage of the contour perimeter
    accuracy = 0.03*cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c,accuracy, True)
    cv2.drawContours(image, [approx], 0, (0,255,0),2)
    cv2.imshow('Approx Poly', image)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Approximating Contours using Convex Hull

In [10]:
# convex hull (drawing a minimum sided polygon around a shap)
image = cv2.imread('hand.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2.imshow('Original Image', image)
cv2.waitKey(0) 

# threshold the image and then find contours 
ret, thresh = cv2.threshold(gray, 176, 255, 0)
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
# sort contors by area and then remove the largest frame contour
n = len(contours) -1
contours = sorted(contours, key=cv2.contourArea, reverse=False)[:n]

# Iterate through contours and draw the convex hull
for c in contours:
    hull = cv2.convexHull(c)
    cv2.drawContours(image, [hull], 0, (0, 255, 0), 2)
    cv2.imshow('Convex Hull', image)

cv2.waitKey(0)    
cv2.destroyAllWindows()

### Shape Matching

In [2]:
# shape matching

template = cv2.imread('shapes.png')
template_gray = cv2.cvtColor(template,cv2.COLOR_BGR2GRAY)
cv2.imshow('Template', template_gray)
cv2.waitKey()

# load the target image to match with the template
target = cv2.imread('star.png')
target_gray = cv2.cvtColor(target,cv2.COLOR_BGR2GRAY)
cv2.imshow('Gray Target', target_gray)
cv2.waitKey()

# threshold both images first before using cv2.findContours
ret, thresh1 = cv2.threshold(template_gray, 127, 255, 0)
ret, thresh2 = cv2.threshold(target_gray, 127, 255, 0)

# first find contours in template
contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

# sort the contours by area
sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)

# we extract the second largest contour which will be our template contour
template_contour = contours[1]

# extract contours from second target image
contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

for c in contours:
    # iterate through each contour in the target image and 
    match = cv2.matchShapes(template_contour, c, 3, 0.0)
    print (match)
    if match < 0.15:
        closest_contour = c
    else:
        closest_contour = [] 
                
cv2.drawContours(target, [closest_contour], -1, (0,255,0), 3)
cv2.imshow('Output', target)
cv2.waitKey()
cv2.destroyAllWindows()

0.08389081634879778
0.09435604747204475


### Shape Identification

In [12]:
# load and then gray scale image

image = cv2.imread('shapes.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2.imshow('Identifying Shapes',image)
cv2.waitKey(0)

ret, thresh = cv2.threshold(gray, 127, 255, 1)

# extract contours
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

for cnt in contours:
    
    # get approximate polygons
    approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt,True),True)
    
    if len(approx) == 3:
        shape_name = "Triangle"
        cv2.drawContours(image,[cnt],0,(0,255,0),-1)
        
        # find contour center to place text at the center
        M = cv2.moments(cnt)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
    
    elif len(approx) == 4:
        x,y,w,h = cv2.boundingRect(cnt)
        M = cv2.moments(cnt)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        
        # check to see if 4-side polygon is square or rectangle
        # cv2.boundingRect returns the top left and then width and 
        if abs(w-h) <= 3:
            shape_name = "Square"
            
            # find contour center to place text at the center
            cv2.drawContours(image, [cnt], 0, (0, 125 ,255), -1)
            cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
        else:
            shape_name = "Rectangle"
            
            # find contour center to place text at the center
            cv2.drawContours(image, [cnt], 0, (0, 0, 255), -1)
            M = cv2.moments(cnt)
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
            
    elif len(approx) == 10:
        shape_name = "Star"
        cv2.drawContours(image, [cnt], 0, (255, 255, 0), -1)
        M = cv2.moments(cnt)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
        
    elif len(approx) >= 15:
        shape_name = "Circle"
        cv2.drawContours(image, [cnt], 0, (0, 255, 255), -1)
        M = cv2.moments(cnt)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)

    cv2.imshow('Identifying Shapes',image)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

### Line Detection

In [35]:
# line detection in images
image = cv2.imread('sudoku.jpg')

# grayscale and canny Edges extracted
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 100, 170, apertureSize = 3)

# run HoughLines using a rho accuracy of 1 pixel
# theta accuracy of np.pi / 180 which is 1 degree
# our line threshold is set to 240 (number of points on line)
lines = cv2.HoughLines(edges, 1, np.pi / 360, 240)

# we iterate through each line and convert it to the format
# required by cv.lines (i.e. requiring end points)
for rho, theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

cv2.imshow('Hough Lines', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Line Detection using Probabilistic Hough Lines 

In [43]:
# line detection using Probabilistic Hough Lines 

# grayscale and Canny Edges extracted
image = cv2.imread('sudoku.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 100, 170, apertureSize = 3)

# Again we use the same rho and theta accuracies
# However, we specific a minimum vote (pts along line) of 100
# and Min line length of 5 pixels and max gap between lines of 10 pixels
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 10, 5, 10)
print (lines.shape)

for x1, y1, x2, y2 in lines[0]:
    cv2.line(image, (x1, y1), (x2, y2),(0, 255, 0), 3)

cv2.imshow('Probabilistic Hough Lines', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(175, 1, 4)


### Blob Detection

In [59]:
# detecting the centers of sunflowers
 
# read the image
image = cv2.imread("sunflowers.jpg")

# set up the detector with default parameters.
detector = cv2.SimpleBlobDetector_create()
 
# detect blobs.
keypoints = detector.detect(image)
 
# draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of
# the circle corresponds to the size of blob
blank = np.zeros((1,1)) 
blobs = cv2.drawKeypoints(image, keypoints, blank, (255,255,255),cv2.DRAW_MATCHES_FLAGS_DEFAULT)
                     
cv2.imshow("Blobs on SunFlowers", blobs)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Counting Circular Shapes

In [93]:
# load image
image = cv2.imread("blobs.png", 0)
cv2.imshow('Original Image',image)
cv2.waitKey(0)

# intialize the detector using the default parameters
detector = cv2.SimpleBlobDetector_create()
 
# detect blobs
keypoints = detector.detect(image)
 
# draw blobs on our image as red circles
blank = np.zeros((1,1)) 
blobs = cv2.drawKeypoints(image, keypoints, blank, (0,255,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                                      
number_of_blobs = len(keypoints)
text = "Total Number of Blobs: " + str(len(keypoints))
cv2.putText(blobs, text, (20, 550), cv2.FONT_HERSHEY_SIMPLEX, 1, (00,255,0), 2)

# display image with blob keypoints
cv2.imshow("Blobs using default parameters", blobs)
cv2.waitKey(0)

# set our filtering parameters
# initialize parameter settiing using cv2.SimpleBlobDetector
params = cv2.SimpleBlobDetector_Params()

# set area filtering parameters
params.filterByArea = True
params.minArea = 100

# set circularity filtering parameters
params.filterByCircularity = True 
params.minCircularity = 0.9

# set convexity filtering parameters
params.filterByConvexity = False
params.minConvexity = 0.2
    
# set inertia filtering parameters
params.filterByInertia = True
params.minInertiaRatio = 0.01

# create a detector with the parameters
detector = cv2.SimpleBlobDetector_create(params)
    
# detect blobs
keypoints = detector.detect(image)

# draw blobs on our image as red circles
blank = np.zeros((1,1)) 
blobs = cv2.drawKeypoints(image, keypoints, blank, (0,255,0),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                                      
number_of_blobs = len(keypoints)
text = "Number of Circular Blobs: " + str(len(keypoints))
cv2.putText(blobs, text, (20, 550), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

# show blobs
cv2.imshow("Filtering Circular Blobs Only", blobs)
cv2.waitKey(0)
cv2.destroyAllWindows()